In [1]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler
import numpy as np
import pandas as pd
import torch as torch

In [69]:
class StaticLayer(nn.Module):
    def __init__(self,in_channels, Trnn, static_features, out_channels = 30, dropout = 0.4):
        super().__init__()
        self.Trnn = Trnn
        self.static_features = static_features
        self.dropout = dropout
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.static = nn.Linear(self.in_channels, self.out_channels)

    def forward(self, x):
        x = x[self.static_features].squeeze(1)
        x = self.dropout(x)
        x = self.static(x)
        return x.unsqueeze(1).repeat(1, self.Trnn, 1)

class ConvLayer(nn.Module):
    def __init__(self, in_channels, timevarying_features, out_channels = 30, kernel_size = 2):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.timevarying_features = timevarying_features

        c1 = nn.Conv1d(self.in_channels, self.out_channels, self.kernel_size, dilation = 1)
        c2 = nn.Conv1d(self.out_channels, self.out_channels, self.kernel_size, dilation = 2)
        c3 = nn.Conv1d(self.out_channels, self.out_channels, self.kernel_size, dilation = 4)
        c4 = nn.Conv1d(self.out_channels, self.out_channels, self.kernel_size, dilation = 8)
        c5 = nn.Conv1d(self.out_channels, self.out_channels, self.kernel_size, dilation = 16)
        c6 = nn.Conv1d(self.out_channels, self.out_channels, self.kernel_size, dilation = 32)

    def forward(self, x):
        x_t = x[self.timevarying_features]
        x_t = x_t.permute(0, 2, 1)
        x_t = F.pad(x_t, (0,0), "constant", 0)
        x_t = c1(x_t)
        x_t = F.pad(x_t, (2,0), "constant", 0)
        x_t = c2(x_t)
        x_t = F.pad(x_t, (4,0), "constant", 0)
        x_t = c3(x_t)
        x_t = F.pad(x_t, (8,0), "constant", 0)
        x_t = c4(x_t)
        x_t = F.pad(x_t, (16,0), "constant", 0)
        x_t = c5(x_t)
        x_t = F.pad(x_t, (32,0), "constant", 0)
        x_t = c6(x_t)
        
        return x_t.permute(0, 2, 1)

class ExpandLayer(nn.Module):
    """Expands the dimension referred to as `expand_axis` into two
    dimensions by applying a sliding window. For example, a tensor of
    shape (1, 4, 2) as follows:

    [[[0. 1.]
      [2. 3.]
      [4. 5.]
      [6. 7.]]]

    where `expand_axis` = 1 and `Trnn` = 3 (number of windows) and
    `lead_future` = 2 (window length) will become:

    [[[[0. 1.]
       [2. 3.]]

      [[2. 3.]
       [4. 5.]]

      [[4. 5.]
       [6. 7.]]]]

    Used for expanding future information tensors

    Parameters
    ----------
    Trnn : int
        Length of the time sequence (number of windows)
    lead_future : int
        Number of future time points (window length)
    expand_axis : int
        Axis to expand"""

    def __init__(self, Trnn, lead_future, future_information, **kwargs):
        super(ExpandLayer, self).__init__(**kwargs)
    
        self.Trnn = Trnn
        self.future_information = future_information
        self.lead_future = lead_future

    def forward(self, x):

        # First create a matrix of indices, which we will use to slice
        # `input` along `expand_axis`. For example, for Trnn=3 and
        # lead_future=2,
        # idx = [[0. 1.]
        #        [1. 2.]
        #        [2. 3.]]
        # We achieve this by doing a broadcast add of
        # [[0.] [1.] [2.]] and [[0. 1.]]
        x = x[self.future_information]
        idx = torch.add(torch.arange(self.Trnn).unsqueeze(axis = 1), 
                        torch.arange(self.lead_future).unsqueeze(axis = 0))
        # Now we slice `input`, taking elements from `input` that correspond to
        # the indices in `idx` along the `expand_axis` dimension
        return x[:, idx, :]


    
class HorizonSpecific(nn.Module):
    def __init__(self, Tpred, Trnn, num = 20):
        super().__init__()
        self.Tpred = Tpred
        self.Trnn = Trnn
        self.num = num
        
    def forward(self, x):
        
        x = nn.Linear(x.size(-1), self.Tpred*self.num)(x)
        x = F.relu(x)

        return x.view(-1, self.Trnn, self.Tpred, 20)

class HorizonAgnostic(nn.Module):
    def __init__(self, out_channels, lead_future):
        super().__init__()
        self.out_channels = out_channels
        self.lead_future = lead_future
        
        
    def forward(self, x):
        x = nn.Linear(x.size(-1), self.out_channels)(x)
        x = F.relu(x)
        x = x.unsqueeze(axis = 2)
        x = x.repeat(1,1, self.lead_future, 1)

        return x
    
class LocalMlp(nn.Module):
    def __init__(self, hidden, output):
        super().__init__()
        self.hidden = hidden
        self.output = output
        self.l2 = nn.Linear(self.hidden, self.output)
        
    def forward(self,x):
        x = nn.Linear(x.size(-1), self.hidden)(x)
        x = F.relu(x)
        x = self.l2(x)
        x = F.relu(x)

        return x


class Span1(nn.Module):
    def __init__(self, Trnn, lead_future, num_quantiles):
        super().__init__()
        self.Trnn = Trnn
        self.lead_future = lead_future
        self.num_quantiles = num_quantiles
        
    def forward(self, x):
        x = nn.Linear(x.size(-1), self.num_quantiles)
        x = F.relu(x.contiguous().view(-1, x.size(-2), x.size(-1)))
        x = x.view(-1, self.Trnn, self.lead_future, self.num_quantiles)
        x = x.view(-1, self.Trnn, self.lead_future*self.num_quantiles)

        return x
    
class SpanN(nn.Module):
    def __init__(self, Trnn, lead_future, num_quantiles, spanN_count):
        super().__init__()
        self.Trnn = Trnn
        self.lead_future = lead_future
        self.num_quantiles = num_quantiles
        self.spanN_count = spanN_count
        
    def forward(self, x):
        x = x.permute(0, 1, 3, 2)
        x = x.contiguous().view(-1, self.Trnn, x.size(-2) * x.size(-1))

        x = nn.Linear(x.size(-1), self.spanN_count * self.num_quantiles)

        return x

In [38]:
class GlobalFutureLayer(nn.Module):
    def __init__(self, lead_future, future_features_count, out_channels = 30):
        super(GlobalFutureLayer, self).__init__()
        self.lead_future = lead_future
        self.future_features_count = future_features_count
        self.out_channels = out_channels
        self.l1 = nn.Linear(self.lead_future*self.future_features_count, self.out_channels)
        
    def forward(self, x):
        x = x.view(-1, self.Trnn, self.lead_future * self.future_features_count)
        
        return self.l1(x)

In [70]:

class MQCNNModel(nn.Module):
    def __init__(self, Trnn, static_features, timevarying_features, future_information, ltsp, lead_future):
        super(MQCNNModel, self).__init__()
        
        self.Trnn = Trnn
        self.static_features = static_features
        self.timevarying_features = timevarying_features
        self.future_information = future_information
        self.ltsp = ltsp
        self.lead_future = lead_future

        self.encoder = MQCNNEncoder(self.Trnn, self.static_features, self.timevarying_features)
        self.decoder = MQCNNDecoder(self.Trnn, self.lead_future, self.ltsp, self.future_information, self.future_information)

    def forward(self, x):
        encoding = self.encoder(x)
        x = self.decoder(encoding, x)
        
        return x

class MQCNNEncoder(nn.Module):
    def __init__(self, Trnn, static_features, timevarying_features):
        super(MQCNNEncoder, self).__init__()
        self.Trnn = Trnn
        self.static_features = static_features
        self.timevarying_features = timevarying_features
        self.static = StaticLayer(in_channels = len(self.static_features),
                                  Trnn = self.Trnn,
                                  static_features = self.static_features)

        self.conv = ConvLayer(in_channels = len(self.timevarying_features),
                             timevarying_features = self.timevarying_features)

    def forward(self, x):
        x_s = self.static(x)
        x_t = self.conv(x)

        return torch.cat((x_s, x_t), axis = 1)


class MQCNNDecoder(nn.Module):
    """Decoder implementation for MQCNN

    Parameters
    ----------
    config
        Configurations
    ltsp : list of tuple of int
        List of lead-time / span tuples to make predictions for
    expander : HybridBlock
        Overrides default future data expander if not None
    hf1 : HybridBlock
        Overrides default global future layer if not None
    hf2 : HybridBlock
        Overrides default local future layer if not None
    ht1 : HybridBlock
        Overrides horizon-specific layer if not None
    ht2 : HybridBlock
        Overrides horizon-agnostic layer if not None
    h : HybridBlock
        Overrides local MLP if not None
    span_1 : HybridBlock
        Overrides span 1 layer if not None
    span_N : HybridBlock
        Overrides span N layer if not None

    Inputs:
        - **xf** : Future data of shape
            (batch_size, Trnn + lead_future - 1, num_future_ts_features)
        - **encoded** : Encoded input tensor of shape
            (batch_size, Trnn, n) for some n
    Outputs:
        - **pred_1** :  Span 1 predictions of shape
            (batch_size, Trnn, Tpred * num_quantiles)
        - **pred_N** : Span N predictions of shape
            (batch_size, Trnn, span_N_count * num_quantiles)

        In both outputs, the last dimensions has the predictions grouped
        together by quantile. For example, the quantiles are P10 and P90
        then the span 1 predictions will be:
        Tpred_0_p50, Tpred_1_p50, ..., Tpred_N_p50, Tpred_0_p90,
        Tpred_1_p90, ... Tpred_N_90


    """

    def __init__(self, Trnn, lead_future, future_information, ltsp, num_quantiles = 2):
        super(MQCNNDecoder, self).__init__()
        self.future_features_count = len(future_information)
        self.future_information = future_information
        self.Trnn = Trnn
        self.lead_future = lead_future
        self.ltsp = ltsp
        self.num_quantiles = num_quantiles

        # We assume that Tpred == span1_count.
        self.Tpred = max(map(lambda x: x[0] + x[1], self.ltsp))
        span1_count = len(list(filter(lambda x: x[1] == 1, self.ltsp)))
        self.spanN_count = len(list(filter(lambda x: x[1] != 1, self.ltsp)))

        self.expander = ExpandLayer(self.Trnn, self.lead_future, self.future_information)
        self.hf1 = GlobalFutureLayer(self.lead_future, self.future_features_count, 30)
        self.ht1 = HorizonSpecific(self.Tpred, self.Trnn, 20)
        self.ht2 = HorizonAgnostic(100, self.lead_future)
        self.h = LocalMlp(50, 10)
        self.span_1 = Span1(self.Trnn, self.lead_future, self.num_quantiles)
        self.span_N = SpanN(self.Trnn, self.lead_future, self.num_quantiles, self.spanN_count)

    def forward(self, F, x, encoded):
        xf = x[self.future_information]
        expanded = self.expander(xf)
        hf1 = self.hf1(expanded)
        hf2 = F.tanh(expanded)

        ht = torch.cat(encoded, hf1, dim=-1)
        ht1 = self.ht1(ht)
        ht2 = self.ht2(ht)
        h = torch.cat(ht1, ht2, hf2, dim=-1)
        h = self.h(h)
        return self.span_1(h), self.span_N(h)



In [76]:
x = pd.DataFrame(torch.rand(1000, 456, 25))
x.columns = cols

ValueError: Length mismatch: Expected axis has 1 elements, new values have 25 elements

In [77]:
x.head()

,0
0,"[[tensor(0.5033), tensor(0.1975), tensor(0.850..."
1,"[[tensor(0.9509), tensor(0.9503), tensor(0.358..."
2,"[[tensor(0.6205), tensor(0.8250), tensor(0.198..."
3,"[[tensor(0.2286), tensor(0.3401), tensor(0.096..."
4,"[[tensor(0.7841), tensor(0.3957), tensor(0.069..."
...,...
995,"[[tensor(0.3264), tensor(0.3802), tensor(0.868..."
996,"[[tensor(0.5456), tensor(0.8882), tensor(0.413..."
997,"[[tensor(0.9219), tensor(0.4395), tensor(0.183..."
998,"[[tensor(0.3078), tensor(0.1553), tensor(0.307..."


In [5]:
cols = []
for i in range(25):
    cols.append(f'col{i}')

In [6]:
static_features = ['col0', 'col1', 'col2', 'col3', 'col4']

timevarying_features = ['col5','col6','col7','col8','col9','col10','col11','col12','col13','col14','col15','col16','col17','col18','col19', 'col20', 'col21', 'col22', 'col23', 'col24', 'col25']

future_information = ['col19', 'col20', 'col21', 'col22', 'col23', 'col24', 'col25']

In [8]:
max(map(lambda x: x[0] + x[1], ltsp))

92

In [9]:
len(list(filter(lambda x: x[1] == 1, ltsp)))

92

In [71]:
m = MQCNNModel(Trnn = 365, 
               lead_future = 91, 
               static_features= static_features, 
               timevarying_features = timevarying_features, 
               future_information = future_information, 
               ltsp = ltsp)

In [72]:
m(x)

IndexError: too many indices for tensor of dimension 3

In [ ]:
x_f = torch.rand(500, 456, 10)

In [ ]:
x_f.shape

In [ ]:
idx = torch.add(torch.arange(365).unsqueeze(axis = 1), torch.arange(91).unsqueeze(axis = 0))

In [ ]:
idx

In [ ]:
expanded = x_f[:, idx, :] # expanded 

In [ ]:
expanded.shape

In [ ]:
expanded.view(500, 365, 910).shape # expand

In [ ]:
hf1 = nn.Linear(910, 30)(x_f[:, idx, :].view(-1, 365, 910))

In [ ]:
hf1.shape

In [ ]:
x = torch.rand(500, 456, 50)

In [ ]:
x_t = x[:, :365, 10:40]
x_f = x[:, :, 40:]

In [ ]:
x_s = torch.rand(500, 10)

In [ ]:
x_s.shape, x_t.shape, x_f.shape

In [ ]:
s = nn.Linear(10, 30)(x_s)


In [ ]:
c1 = F.pad(x_t.permute(0, 2, 1), (1, 0), "constant", 0)
c1 = nn.Conv1d(30, 30, 2, dilation = 1)(c1)
c2 = F.pad(c1, (2, 0), "constant", 0)
c2 = nn.Conv1d(30, 30, 2, dilation = 2)(c2)
c3 = F.pad(c2, (4, 0), "constant", 0)
c3 = nn.Conv1d(30, 30, 2,  dilation = 4)(c3)
c4 = F.pad(c3, (8, 0), "constant", 0)
c4 = nn.Conv1d(30, 30, 2, dilation = 8)(c4)
c5 = F.pad(c4, (16, 0), "constant", 0)
c5 = nn.Conv1d(30, 30, 2, dilation = 16)(c5)
c6 = F.pad(c5, (32, 0), "constant", 0)
c6 = nn.Conv1d(30, 30, 2, dilation = 32)(c6)


In [ ]:
t = c6.permute(0, 2, 1)

In [ ]:
s = s.unsqueeze(1).repeat(1, 365, 1)

In [ ]:
t.shape

In [ ]:
encoding = torch.cat((s, t), axis = 2)

In [ ]:
encoding.shape

In [ ]:
ht = torch.cat((hf1, encoding), axis = 2)

In [ ]:
ht.shape

In [ ]:
nn.Linear(90, 1820)(ht).shape

In [ ]:
ht1 = nn.Linear(90, 1820)(ht).view(-1, 365, 91, 20)

In [ ]:
ht2 = nn.Linear(90, 100)(ht).unsqueeze(axis = 2).repeat(1, 1, 91, 1)

In [ ]:
hf2 = F.relu(expanded)

In [ ]:
expanded.shape, ht1.shape, ht2.shape

In [ ]:
h = torch.cat((ht1, ht2, hf2), dim = -1)

In [ ]:
class local_mlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(130, 50)
        self.l2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x  = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        
        return x

In [ ]:
l = local_mlp()

In [ ]:
h2 = l(h)

In [ ]:
h3 = nn.Linear(10, 2)(h2)

In [ ]:
y = F.leaky_relu(h2.contiguous().view(-1, h3.size(-2), h3.size(-1)))

In [ ]:
y.view(-1, 365, 91, 2).view(500, 365, 182).shape

In [ ]:
h.shape

In [ ]:
h4 = h2.permute(0, 1, 3, 2).contiguous().view(-1, 365, h2.size(-2) * h2.size(-1))

In [ ]:
nn.Linear(h4.size(-1), 40)(h4).shape

In [ ]:
class Dataset(torch.utils.data.DataLoader):
    
    def __init__(self, data, static_features, time_varying_known, time_varying_unknown, targets):
        
        self.targets = targets
        self.data = data
        self.static = static_features
        self.tv_known = time_varying_known
        self.tv_unknown = time_varying_unknown
        
    def __len__(self):
        
        return len(self.features['time_idx'].unique())
    
    def __getitem__(self, index):
        
        print(self.features)
        
        X_static = self.data.loc[self.features['time_idx'] == index, [self.static]]
        X_tv_known = self.data.loc[self.features['time_idx'] == index, [self.tv_known]]
        X_tv_unknown = self.data.loc[self.features['time_idx'] == index, [self.tv_unknown]]
        Y = self.targets.loc[self.targets['time_idx'] == index]
        
        return X, Y

In [ ]:
x = torch.rand(500, 456, 50)
y = x[:, :, 50]
static = x[:, 0, :10]
tv_known = x[:, 10:20]
tv_unknown = [20:50]
d = Dataset(x, static, tv_known, tv_known)

In [ ]:
torch.rand(500, 1, 10).squeeze(1).shape